In [1]:
import numpy as np
import pandas as pd


ModuleNotFoundError: No module named 'numpy'

In [3]:
sammple_hist = np.load(r"C:\Users\tskor\Documents\data\histograms\60_buckets_9_bands_60_res\60\Accomack_51\2017\5-6.npy")


In [3]:
d = {'m5':5, 'm6':6}

print('unsorted: ', d, 'sorted: ',  sorted(d))

unsorted:  {'m5': 5, 'm6': 6} sorted:  ['m5', 'm6']


In [16]:
x = sammple_hist.reshape((9,-1))

In [17]:
for i in range(2):
    print(x[i,:])

[   0    0    0    0    0    0    0   14  638 2325 4123 6452 7469 8516
 6163 4186 2190 1048  641  421  377  220  124   94   58   49   43   62
   52   35   29   17    8   14    4    3    3    3    5    4    3    0
    0    2    0    5    5    4    7   11    4    9    4    2    3    2
    0    3    1    2]
[   0    0    0    0    0    0    0    0    0  596 4350 6808 7708 9471
 7134 4728 2060  879  590  378  170   92   44   59   51   46   29   53
   69   29   12    5    6    5    5    2    1    2    5    2    4    2
    2    4    2    3    5    8    8    7    9    2    2    4    0    0
    3    2    2    0]


In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")

print(dataset)


False

In [25]:
import numpy as np
import os
INPUT_PATH = r'C:\Users\tskor\Documents\data\histograms\histograms_county_year'
HISTOGRAM_NAME = 'histogram-31035-2017.npy'

In [26]:
data_to_investigate = np.load(os.path.join(INPUT_PATH, HISTOGRAM_NAME), allow_pickle=True)

#### Test multimodal LSTM

In [2]:
# Add this at the beginning of your notebook
%load_ext autoreload
%autoreload 2  
# Automatically reload all modules before executing code

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import sys
#  Add the root directory to the path
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(root_dir)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import numpy as np

# Make sure eager execution is enabled
tf.config.run_functions_eagerly(True)

In [4]:
#Dataset
from data.test_train_split import test_train_split_multi_modal
from models.lstm import LstmWeather, PreprocessingHead, Embeddings

WEATHER_PATH = r"C:\Users\tskor\Documents\GitHub\inovation_project\4_Data_Sample\weather_data-adams-2017-CO-8001.csv"
HISTOGRAM_PATH = r"C:\Users\tskor\Documents\GitHub\inovation_project\4_Data_Sample\histogram-08001-2017_reduced.csv"
LABELS_PATH = r"C:\Users\tskor\Documents\GitHub\inovation_project\4_Data_Sample\combined_labels_with_fips.npy"

CAT_FEATURES = ["fips"]

In [5]:
datasets_monthly_train, datasets_monthly_test = test_train_split_multi_modal(
    WEATHER_PATH,
    HISTOGRAM_PATH,
    LABELS_PATH,
    index=["fips", "year"],
    months=[5, 7, 9],
)
histograms_train, weather_train = datasets_monthly_train

histogram data shape: (3, 11)
weather data shape: (3, 15)


In [6]:
histograms_train[0]


,fips,year,histogram_200,histogram_201,histogram_202,histogram_203,histogram_204,histogram_205,histogram_206,histogram_207,histogram_208,histogram_209,month
0,08001,2017,2059,900,214,86,72,43,43,41,27,18,5


In [33]:
preproc_histogram = PreprocessingHead(
    histograms_train[0], CAT_FEATURES
)




In [34]:
input = {col: histograms_train[0][col].values for col in histograms_train[0].columns}

In [35]:
preproc_histogram(input)

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [38]:
preproc_histogram.summary()


Model: "preprocessing_head_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_6 (Normalization) │ (12)                   │            25 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_12 (Concatenate)    │ (12)                   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_13 (Concatenate)    │ (1, 14)                │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ string_lookup_6 (StringLookup)  │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25 (104.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 25 (104.00 B)